# Utilizzo del Item-Based Collaborative Filtering

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd

In [2]:
def cosin_similarity(x, y): 
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):
        if np.isnan(i) or np.isnan(j):
          continue
        t1+=i*j
        t2+=i*i
        t3+=j*j
    return t1/(np.sqrt(t2) * np.sqrt(t3))

In [3]:
# lettura dei rating normalizzati dal relativo CSV
pd.set_option('display.max_columns', 500)
normalized_ratings_df = pd.read_csv('./data/normalized_ratings.csv')
# lettura dei rating non normalizzati dal relativo CSV
ratings_df = pd.read_csv('./data/jester_jokes_ratings.csv')
# se si calcola la similarità tra soli due item corated viene il giusto risultato
# altrimenti i 0 vengono considerati, che facciamo?
#x = normalized_ratings_df.iloc[0, 79:80]
#y = normalized_ratings_df.iloc[1, 79:80]

# la funzione weight_factor prende in input due array quindi ci vanno le [] esterne ad iloc
x = [normalized_ratings_df.iloc[0, 79]]
y = [normalized_ratings_df.iloc[1, 79]]
sim = cosin_similarity(x,y)
print(x)
print(y)
print(sim)
#normalized_ratings_df.head(2)

[-5.988108108]
[6.2337]
-1.0


## Separazione dei rating normalizzati

Si separano i rating noramlizzati in due parti: 
1. una parte contenente tutti gli utenti che hanno valutato tutte e 100 le barzellette;
2. una parte contenente tutti gli utenti che non hanno valutato tutte le barzellette.

Si seleziona poi l'utente target (al quale si vuole fare una raccomandazione) dalla matrice sparsa e si cercano i vicini di quell'utente nella la matrice non sparsa.

In [4]:
# utenti che hanno valutato tutte e 100 le barzellette
complete_ratings = normalized_ratings_df[normalized_ratings_df['number_of_jokes_rated'] == 100]
print('totale utenti che hanno valutato tutte le barzellette: ', len(complete_ratings))

# utenti che non hanno valutato tutte le barzellette
sparse_ratings = normalized_ratings_df[normalized_ratings_df['number_of_jokes_rated'] != 100]
print('totale utenti che non hanno valutato tutte le barzellette: ', len(sparse_ratings))

totale utenti che hanno valutato tutte le barzellette:  7200
totale utenti che non hanno valutato tutte le barzellette:  17783


In [5]:
# si sceglie un utente target e si restituisce una lista degli indici dei 
# joke che l'utente non ha valutato
def lista_of_index_no_vote(target):
    x = normalized_ratings_df.iloc[target-1, 2:].tolist()
    lista_indici = []
    for i in range(len(x)):  
        if np.isnan(x[i]):
            lista_indici.append(i+1)
    return lista_indici

def lista_of_index_vote(target):
    x = normalized_ratings_df.iloc[target-1, 2:].tolist()
    lista_indici = []
    for i in range(len(x)):  
        if not(np.isnan(x[i])):
            lista_indici.append(i+1)
    return lista_indici

# per ogni joke non votato si va a predire un valore
def neighbor(target):
    l = lista_of_index_no_vote(target)
    k = lista_of_index_vote(target)
    print(l, k)
    similarity = []
    predizioni = []
    joke_pred_2_sim_vote = {}
    # per ogni joke non votato 
    for elem in l: 
        #similarity = []
        # per ogni joke votato 
        sommatoria1, sommatoria2 = 0, 0
        for i in k:
            id = "joke_" + str(i)
            # colonna di cui dare la predizione
            x = normalized_ratings_df.iloc[:, elem+1].tolist()
            # si prendono tutte le righe (:) della colonna i+1
            y = normalized_ratings_df.iloc[:, i+1].tolist()
            sim = cosin_similarity(x, y)
            #print(("joke_"+str(elem),id, float(sim)))
            if "joke_"+str(elem) not in joke_pred_2_sim_vote:
                joke_pred_2_sim_vote["joke_"+str(elem)] = [(id, float(sim), normalized_ratings_df.iloc[target-1, i+1])] 
            else: 
                joke_pred_2_sim_vote["joke_"+str(elem)].append((id, float(sim), normalized_ratings_df.iloc[target-1, i+1]))
                
            #similarity.append(("joke_"+str(elem), id, float(sim), normalized_ratings_df.iloc[target-1, i+1]))
            
        #similarity = sorted(similarity, key=lambda x: x[2], reverse=True)
        print("elemento da predire " + str(elem) + "\n")
        # si tengono solo i joke più simili
        #print(similarity[1:4])
        #print("\n")
    return joke_pred_2_sim_vote

#def pred(lista): 
#    sommatoria1 += sim * normalized_ratings_df.iloc[target-1, i+1]
    #print(normalized_ratings_df.iloc[target-1, i+1])
#    sommatoria2 += sim 
#    print(("joke_"+str(elem), sommatoria1/sommatoria2))

In [6]:
def pred(target): 
    d = neighbor(target)
    for elem in d: 
        more_similar = sorted(d[elem], key=lambda x: x[1], reverse=True)
        d[elem] = more_similar[0:5]
        print(elem)
        print(d[elem])
        sommatoria1, sommatoria2 = 0, 0
        for t in d[elem]: 
            sommatoria1 += t[1] * t[2]
            sommatoria2 += t[1] 
        print("\n")
        print(("joke_"+str(elem), sommatoria1/sommatoria2))
        print("\n")

In [7]:
# esempio dove lo user target è id=5
pred(5)

[71, 72, 73, 74, 75, 76, 78, 79, 80] [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 77, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
elemento da predire 71

elemento da predire 72

elemento da predire 73

elemento da predire 74

elemento da predire 75

elemento da predire 76

elemento da predire 78

elemento da predire 79

elemento da predire 80

joke_71
[('joke_90', 0.25508515600516224, -0.292637363), ('joke_58', 0.2476079932972767, -12.42263736), ('joke_84', 0.2354825444408173, 4.0273626369999995), ('joke_20', 0.22035763204153272, 1.257362637), ('joke_63', 0.21301743341451426, 1.457362637)]


('joke_joke_71', -1.3782629902654318)


joke_72
[('joke_89', 0.3429069991844835, -0.432637363), ('joke_93', 0.29296310844764945, 1.017362